# Lab 00f: Hello World ML

Your first machine learning model - a simple spam classifier in under 50 lines!

## Learning Objectives
- Understand the 4-step ML workflow: Load → Train → Predict → Evaluate
- Build a working classifier with scikit-learn
- Know what accuracy, precision, and recall mean

In [ ]:
#@title Install dependencies (Colab only)
#@markdown Run this cell to install required packages in Colab

%pip install -q numpy scikit-learn

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

print("✅ Libraries loaded!")

## Step 1: Load Data

Our dataset has messages labeled as spam (1) or not spam (0):

In [ ]:
# Sample spam and not-spam messages
MESSAGES = [
    # Spam (label = 1)
    "FREE MONEY! Click now to claim your prize!",
    "URGENT: Your account has been compromised!",
    "Congratulations! You've won $1,000,000!",
    "Click here for FREE iPhone!",
    "WINNER! Cash prize waiting!",
    "FREE FREE FREE! Don't miss this!",
    "Urgent action required!",
    "You've won a FREE vacation!",
    "AMAZING DEAL! Get rich quick!",
    "Claim your prize money now!",
    # Not spam (label = 0)
    "Hey, want to grab lunch tomorrow?",
    "Meeting moved to 3pm, see you there",
    "Can you review the document?",
    "Thanks for your help",
    "The report is ready for review",
    "Let's schedule a call for next week",
    "Please find the attached invoice",
    "Looking forward to the conference",
    "Here's the quarterly update",
    "Can we discuss the budget?",
]

LABELS = [1]*10 + [0]*10  # First 10 spam, last 10 not spam
SPAM_WORDS = ["free", "win", "click", "urgent", "money", "prize", "congratulations"]

print(f"📊 Loaded {len(MESSAGES)} messages")
print(f"   Spam: {sum(LABELS)}, Not spam: {len(LABELS) - sum(LABELS)}")

## Understanding the ML Workflow

Before we code, understand what we're building:

```
"FREE MONEY NOW!"     →  [3]  →  Model  →  1 (SPAM)
       ↓                   ↓
   Count spam words    Feature vector
```

**Key Insight**: ML models need numbers, not text. Our "feature extraction" step converts text to numbers.

### The 4-Step Workflow

```
┌─────────────────────────────────────────────────────────────┐
│                    ML WORKFLOW                               │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   1. LOAD DATA        2. TRAIN MODEL                        │
│   ────────────        ─────────────                         │
│   Load examples       Let algorithm                         │
│   with labels         learn patterns                        │
│                                                             │
│         │                   │                               │
│         ▼                   ▼                               │
│                                                             │
│   3. PREDICT          4. EVALUATE                           │
│   ──────────          ────────────                          │
│   Apply model         Measure how                           │
│   to new data         well it works                         │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

## Step 2: Feature Extraction

Convert text to numbers by counting spam indicator words:

In [ ]:
def extract_features(message: str) -> list:
    """
    Convert text to numbers by counting spam words.

    Why this works:
    - Spam emails use urgency words: "FREE", "URGENT", "WIN"
    - Normal emails rarely stack these words
    - More spam words = higher probability of spam
    """
    message_lower = message.lower()
    spam_word_count = sum(1 for word in SPAM_WORDS if word in message_lower)
    return [spam_word_count]

# Test it
print("Testing feature extraction:")
print(f"  'FREE MONEY! Click NOW!' → {extract_features('FREE MONEY! Click NOW!')}")
print(f"  'Meeting at 3pm tomorrow' → {extract_features('Meeting at 3pm tomorrow')}")

## Step 3: Prepare Data

Extract features from all messages and split into training/testing sets:

**Why split?** If we train AND test on the same data, the model "cheats" by memorizing answers.

In [ ]:
# Extract features from all messages
features = [extract_features(msg) for msg in MESSAGES]
X = np.array(features)  # Features (inputs)
y = np.array(LABELS)    # Labels (what we want to predict)

# Split: 80% training, 20% testing
# random_state=42 makes the split reproducible
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,      # 20% for testing
    random_state=42     # Reproducible split
)

print(f"📊 Data prepared:")
print(f"   Training samples: {len(X_train)}")
print(f"   Test samples: {len(X_test)}")

## Step 4: Train the Model

Use Logistic Regression - simple and effective for binary classification:

| Property | Why It's Good for Beginners |
|----------|----------------------------|
| Simple | Only one line to train |
| Fast | Milliseconds on small data |
| Interpretable | Can see feature weights |
| Works | Good baseline accuracy |

In [ ]:
# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

print("✅ Model trained!")
print(f"   What it learned: {model.coef_[0][0]:.2f} weight for spam word count")
print(f"   Interpretation: More spam words → More likely spam")

## Step 5: Make Predictions

Apply the trained model to the test data:

In [ ]:
# Make predictions on test data
predictions = model.predict(X_test)

print("🔮 Predictions made!")
print(f"   Test features: {X_test.flatten()}")
print(f"   Predictions:   {predictions}")
print(f"   Actual labels: {y_test}")

## Step 6: Evaluate the Model

How do we know if our model is good? Use the **confusion matrix**:

```
┌─────────────────────────────────────────────────────────┐
│                    CONFUSION MATRIX                     │
├─────────────────────────────────────────────────────────┤
│                      Predicted                          │
│                  SPAM      NOT SPAM                     │
│              ┌──────────┬──────────┐                    │
│   Actual     │          │          │                    │
│   SPAM       │    TP    │    FN    │ → Recall           │
│              ├──────────┼──────────┤                    │
│   NOT SPAM   │    FP    │    TN    │                    │
│              └──────────┴──────────┘                    │
│                    ↓                                    │
│               Precision                                 │
├─────────────────────────────────────────────────────────┤
│  Accuracy  = (TP + TN) / Total     Overall correctness  │
│  Precision = TP / (TP + FP)        "How many predicted  │
│                                     spam were correct?" │
│  Recall    = TP / (TP + FN)        "How many real spam  │
│                                     did we catch?"      │
└─────────────────────────────────────────────────────────┘
```

### Which Metric Matters?

| Use Case | Focus On | Why |
|----------|----------|-----|
| General | Accuracy | Overall correctness |
| Spam filter | Precision | Don't block legitimate emails |
| Security alert | Recall | Don't miss real threats |

In [ ]:
# Calculate metrics
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)

print("📈 Model Evaluation:")
print(f"   Accuracy:  {accuracy:.1%} (overall correctness)")
print(f"   Precision: {precision:.1%} (of predicted spam, how many were actually spam)")
print(f"   Recall:    {recall:.1%} (of actual spam, how many did we catch)")

## 🎉 Congratulations!

You just built your first ML model! Here's what you learned:

1. **ML workflow**: Load → Extract Features → Split → Train → Predict → Evaluate
2. **Features matter**: What you measure determines what the model learns
3. **Always split data**: Test on unseen data to measure true performance
4. **Understand metrics**: Accuracy isn't everything; consider precision and recall

## 🚀 Bonus Challenges

### Challenge 1: Add More Features

```python
def extract_features_v2(message: str) -> list:
    message_lower = message.lower()
    return [
        sum(1 for word in SPAM_WORDS if word in message_lower),  # Spam word count
        len(message),                    # Message length
        message.count("!"),              # Exclamation marks
        sum(1 for c in message if c.isupper()) / max(len(message), 1),  # Caps ratio
    ]
```

### Challenge 2: Try Different Models

```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Random Forest (usually better)
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Decision Tree (more interpretable)
model = DecisionTreeClassifier(random_state=42)
```

## Next Steps

Continue to:
- **Lab 01**: Build a real phishing classifier with TF-IDF
- **Lab 02**: Learn unsupervised learning (clustering)
- **Lab 03**: Detect anomalies in network data